In [1]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import process
import re
import spacy
import operator
from fuzzywuzzy import fuzz
import operator
from collections import OrderedDict
import time
import databasesqlmat
from databasesqlmat import respond
import sqlite3
conn = sqlite3.connect("sql1.db")
cur = conn.cursor()

C:\Users\mayan\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
intity_design = pd.read_json("doc16.json")
intity_design.head()

,intent_model
0,"{'context': 'Doctor', 'utterances': ['Who are ..."
1,"{'context': 'Patient', 'utterances': ['what is..."


In [3]:
data_frame = pd.json_normalize(intity_design["intent_model"])
data_frame.head()

,context,utterances
0,Doctor,"[Who are the best doctors for heart disease, W..."
1,Patient,[what is the minium age of the patient of doct...


In [4]:
data_frame["utterances"][0] = [i.lower() for i in data_frame["utterances"][0]]
data_frame["utterances"][1] = [j.lower() for j in data_frame["utterances"][1]]

## Context Understanding

In [5]:
class clean:
    def lowerlettercleanword(sentence):
        sentences = [word for sent in sentence for word in sent]
        return sentences

In [6]:
wx = clean.lowerlettercleanword(data_frame["utterances"])
print(wx)


['who are the best doctors for heart disease', 'who are the best doctors for cardiovascular disease', 'who are the best doctors for skin', 'best doctors for dermatology', 'best physician', 'what is the clinic location of doctor mayank', 'best doctor near my location for cardiology', 'please provide the name of the doctors which have fees less than 500 rupees', 'what is timing for doctor soumya', 'which is the best dental clinic', 'what is the fees of doctor rishabh', 'doctor with  fees more than 1000 rupees', 'what is the speciliatity of doctor piyush', 'what is the speciliatity of doctor mayank', 'what is the speciliatity of doctor soumya', 'what is the speciliatity of doctor rishabh', 'what is the speciliatity of doctor deepshikha', 'what is the clinic location of doctor piyush', 'what is the clinic location of doctor soumya', 'what is the clinic location of doctor deepshikha', 'what is the fees of doctor piyush', 'what is the fees of doctor mayank', 'what is the fees of doctor soumy

In [7]:
def getContext(match_string):
    match_string = match_string.lower()
    for idx, i in enumerate(data_frame['utterances']):
        if match_string in i:
            return data_frame['context'][idx]
    return None

In [8]:
print(getContext('Who are the best doctors for skin'))

Doctor


__________________________________

### Name Entity extraction

In [9]:
nlp = spacy.load('en_core_web_sm')

In [10]:
def get_name(doc):
    if doc.ents:
        for ents in doc.ents:
            if ents.label_=='PERSON':
                name = ents.text
                return name
            return None
    else:
        return None

In [11]:
doctors =["Piyush Singh","Soumya","Mayank Mishra","Deepshikha Mishra","Rishabh Yadav","Piyush","SOUMYA","Mayank","Deepshikha","piyush","soumya","mayank","deepshikha","rishabh"]

def get_name1(query):
    query = query.split()
    for i in query:
        for j in doctors:
            if i==j:
                name = j
                return name
            

In [12]:
print(get_name(nlp("Jeff Bezos is the CEO of Amazon")))

Jeff Bezos


In [13]:
print(get_name(nlp("Doctor Soumya  Speiliaty")))

Soumya


_____________________________

### Best recommendation and exact match string

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
stopwords_list = stopwords.words('english')
vectorizer = TfidfVectorizer(max_features=3000, stop_words=stopwords_list)
X = vectorizer.fit_transform(wx)
count_dense = X.todense()

In [15]:
test_query = vectorizer.transform(["Best cardiologist"])
cosine_similarity(test_query, count_dense[1,:])[0][0]

0.42189892440665977

In [16]:
def getContext_details(input_query):
    input_query_vector = vectorizer.transform([input_query])
    max_score = 0
    best_match = None
    context = None
    for ind, vector in enumerate(count_dense):
        sim_score = cosine_similarity(input_query_vector, count_dense[ind,:])[0][0]
        if sim_score>=max_score:
            max_score = sim_score
            best_match = wx[ind]
            context = getContext(best_match)
    return max_score, best_match, context


In [17]:
print(getContext_details("Cardiologist clinic location near me "))

(0.6039712557449533, 'best doctor near my location for cardiology', 'Doctor')


In [18]:
print(getContext_details("what disease is garima suffering from"))

(0.7916980519849912, 'disease name of patient garima', 'Patient')


In [19]:
print(getContext_details("fees of doctor mayank"))

(1.0, 'what is the fees of doctor mayank', 'Doctor')


In [20]:
def patient_name(query):
    name = cur.execute('SELECT Patient_Name FROM Patient WHERE Patient_id=?', (query,))
    for i in name:
        return i[0]
        
            

In [21]:
class pat_id_lst:
    def id_checker_lst(pat_id):
        pat_id_lst = []
        for i in pat_id:
            for j in range(len(i)):
                pat_id_lst.append(i[j])
                
        return pat_id_lst


In [22]:
pat_id = cur.execute("SELECT Patient_id FROM Patient")
print(pat_id_lst.id_checker_lst(pat_id))

[1, 2, 3, 4, 5]


In [23]:
def regrex(query):
    for i in query:
        query = re.sub("dr","doctor",query)
        query = re.sub("Dr","doctor",query)
        query = re.sub("DR","doctor",query)
        query = re.sub("disease","health issue",query)
        query = re.sub("sickness","health issue",query)
        query = re.sub("illness","health issue",query)
        query = re.sub("infection","health issue",query)
        query = re.sub("Illness","health issue",query)
        query =  re.sub("Disease","health issue",query)
        query = re.sub("Sickess","health issue",query)
        query = re.sub("Charges","fees",query)
        query = re.sub("charges","fees",query)
        query = re.sub("wage","fees",query)
        query = re.sub("Wages","fees",query)
        query = re.sub(r"won\'t", "will not", query)
        query = re.sub(r"can\'t","can not",query)
        query = re.sub(r"n\'t", "not", query)
        query= re.sub(r"\'re", "are", query)
        query = re.sub(r"\'s", "is", query)
        query = re.sub(r"\'d", "would", query)
        query = re.sub(r"\'ll", "will", query)
        query = re.sub(r"\'t", "not", query)
        query = re.sub(r"\'ve", "have", query)
        query = re.sub(r"\'m", "am", query)
        query= re.sub("[?]+","",query)
        
    return query    

In [24]:
class bestrecommend:
    def getContext_bestrecommend(input_query):
        input_query = input_query.lower()
        input_query_vector = vectorizer.transform([input_query])
        unsorted_score = []
        unsorted_match = []

        for ind, vector in enumerate(count_dense):
            sim_score = cosine_similarity(input_query_vector, count_dense[ind,:])[0][0]
            if sim_score>=0.4:
                unsorted_score.append(sim_score)
                unsorted_match.append(wx[ind])
        zipped_pairs = zip(unsorted_score,unsorted_match)
        best_recom = [x for _, x in sorted(zipped_pairs,reverse = True)]
        
        return best_recom[:3]    

In [25]:
print(bestrecommend.getContext_bestrecommend("doctor piyush clinic location"))

['what is the clinic location of doctor piyush', 'what is the clinic location of doctor soumya', 'what is the clinic location of doctor mayank']


In [26]:
respond.sql_answer("what is the fees of doctor mayank")


consultation charges of Doctor Mayank is 500 Rupees


In [27]:
getContext_details("illness issue with patient with id 2")

(0.7701282686581568, 'health issue with patient id 4 patient amit', 'Patient')

In [28]:
_,best_match,_ = getContext_details("illness issue with patient harsh with id 2")
respond.sql_answer(best_match)

Heart Attack is the Health issue with  Patient Harsh Mishra


_____________________

## Chatbot desigining

In [29]:
Doctors = ["Piyush Singh","Soumya","Mayank Mishra","Deepshikha Mishra","Rishabh Yadav"]
Patients = ["Aditya Sonkar","Harsh Mishra","Yuvraj Sinha","Amit","Garima"]

In [30]:
flag = True
context = None
Bestmatch = None
reply = None

In [46]:
x = input()
chat(x)

 hi


Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > doctor piyush location


Loading formatted geocoded file...
Clinic location of Doctor Piyush is Pune
Bot > Thanks for Searching
Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > aditya sonkar death reason


...

User > what is the health issue with harsh


Heart Attack is the Health issue with  Patient Harsh Mishra
Please provide query to search again or type exit to end the Conversation
Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > 


Bot > Please provide the Input
Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > exit


In [45]:
def chat(x):
    flag = True
    context = None
    Bestmatch = None
    reply = None
    while flag:
        Yes_check = 'Yes'
        No_check = 'No'
        query = x
        query = query.strip()
        query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
        query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
    
        query = regrex(query)
        print("Bot > Hello, I am a bot design for Specific Purpose How can I help you?")
        time.sleep(1)
        print("Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating")
        time.sleep(1)
        print("Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor")
        time.sleep(1)
        print("Bot > Here is the list of Doctors {}".format(Doctors))
        time.sleep(1)
        print("Bot > Here is the list of Patients {}".format(Patients))
        time.sleep(1)
        print("Bot > Please Provide your Query for Search")
        query = input("User >")
        query = regrex(query)

        score,_,context = getContext_details(query)
    
        if len(query)==0:
            print("Bot > Please provide the Input")
        elif query=="exit":
            flag=False
    
        else:
            if(score<0.4):
                print("Bot > I am a bot design for Database related issues, Please Provide Suitable Query")
            
            elif(score>=.80):
                _,best_match,_ = getContext_details(query)
                answer = respond.sql_answer(best_match)
                print("Bot > Thanks for Searching")
            
            
            
            
            elif(score>=0.4 and context == "Doctor"):
                print("Bot > You want to check Detail Information about Doctor {} Yes or No" .format(get_name1((query))))
                query =input("User >")
                query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                query = regrex(query)
                if(query == "exit"):
                    flag = False
                if (query.casefold()==Yes_check.casefold()):
                    print("Bot > Please provide your search query again for confirmation")
                    query = input("User >")
                    query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                    query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                    query = regrex(query)
                    if(query == "exit"):
                        flag = False
                    else:
                        Bestmatch = bestrecommend.getContext_bestrecommend(query)
                        for response in Bestmatch:
                            print("Bot > {} ".format(response))
                        print(" ")
                        print("Bot > None of the Above")
                        print(" ")
                        print("Bot > Above are the matching query,Select the suitable query")
                        print("Bot > Enter the best suitable query")
                        query = input("User >")
                        query = query.lstrip()
                        query = query.rstrip()
                        query = query.lstrip("@,<,>,#,$,%,%,^,&,*")
                        query = regrex(query)
                        if (query=="exit"):
                            flag=False
                        elif(query == "None of the Above"):
                            print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                            break
                        else:
                            respond.sql_answer(query)
                            time.sleep(2)
                        
                
                
                else:
                    print("bot > Sorry, I am currently unavailable to find query")
                    flag = False
                
            elif(score>=0.4 and context=="Patient"):
                print("Bot > Paient Informations are very Sensitive Information to check the information \n Please Provide the Patient id")
                time.sleep(2)
            
                query = input("User >")
                query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
                query = query.lstrip()
                query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
                query = query.rstrip()
            
                try:
                    query = int(query)
                
                
                    print("Bot > You want to check Detail Information about Patient {} Yes or No" .format(patient_name(query)))   
                    query = input("User >")
                    query = regrex(query)
                    if (query=="exit"):
                        flag = False
                    if (query.casefold()==Yes_check.casefold()):
                        print("Bot > Please provide your Search query again for confirmation")
                        query = input("User >")
                        query = regrex(query)
                        if (query == "exit"):
                            flag = False
                        else:
                            Bestmatch_1 = bestrecommend.getContext_bestrecommend(query)
                            for response in Bestmatch_1:
                                print("Bot > {}".format(response))
                                print(" ")
                            print("Bot > None of the Above") 
                            print(" ")
                            print("Bot > Above are the matching query,Select the suitable query")
                            print("Bot > Enter the best suitable query")
                            query = input("User >")
                            query = query.lstrip()
                            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
                            query = query.rstrip()
                            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
                            query = regrex(query)
                            if (query=="exit"):
                                flag = False
                    
                            elif(query == "None of the Above"):
                                print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                                break
                    
                            else:
                                answer = respond.sql_answer(query)
                                print("Please provide query to search again or type exit to end the Conversation")    
                
              
                    elif(query.casefold()==No_check.casefold()):
                        print("bot > Sorry, I am currently unavailable to find query")
                        print("Please provide query to search again or type exit to end the Conversation")
                except:
                    print("Bot > You are providing incorrect input")
                
                
            

    
    

__________________________________________________________

In [31]:
while flag:
    Yes_check = 'Yes'
    No_check = 'No'
    query =input("User >")
    query = query.strip()
    query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
    query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
    
    query = regrex(query)
    print("Bot > Hello, I am a bot design for Specific Purpose How can I help you?")
    time.sleep(1)
    print("Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating")
    time.sleep(1)
    print("Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor")
    time.sleep(1)
    print("Bot > Here is the list of Doctors {}".format(Doctors))
    time.sleep(1)
    print("Bot > Here is the list of Patients {}".format(Patients))
    time.sleep(1)
    print("Bot > Please Provide your Query for Search")
    query = input("User >")
    query = regrex(query)

    score,_,context = getContext_details(query)
    
    if len(query)==0:
        print("Bot > Please provide the Input")
    elif query=="exit":
        flag=False
    
    else:
        if(score<0.4):
            print("Bot > I am a bot design for Database related issues, Please Provide Suitable Query")
            
        elif(score>=.80):
            _,best_match,_ = getContext_details(query)
            answer = respond.sql_answer(best_match)
            print("Bot > Thanks for Searching")
            
            
            
            
        elif(score>=0.4 and context == "Doctor"):
            print("Bot > You want to check Detail Information about Doctor {} Yes or No" .format(get_name1((query))))
            query =input("User > ")
            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
            query = regrex(query)
            if(query == "exit"):
                flag = False
            if (query.casefold()==Yes_check.casefold()):
                print("Bot > Please provide your search query again for confirmation")
                query =input("User >")
                query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                query = regrex(query)
                if(query == "exit"):
                    flag = False
                else:
                    Bestmatch = bestrecommend.getContext_bestrecommend(query)
                    for response in Bestmatch:
                        print("Bot > {} ".format(response))
                        print(" ")
                    print("Bot > None of the Above")
                    print(" ")
                    print("Bot > Above are the matching query,Select the suitable query")
                    print("Bot > Enter the best suitable query")
                    query = input("User >")
                    query = query.lstrip()
                    query = query.rstrip()
                    query = query.lstrip("@,<,>,#,$,%,%,^,&,*")
                    query = regrex(query)
                    if (query=="exit"):
                        flag=False
                    elif(query == "None of the Above"):
                        print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                        break
                    else:
                        respond.sql_answer(query)
                        time.sleep(2)
                        
                
                
            else:
                print("bot > Sorry, I am currently unavailable to find query")
                flag = False
                
        elif(score>=0.4 and context=="Patient"):
            print("Bot > Paient Informations are very Sensitive Information to check the information \n Please Provide the Patient id")
            time.sleep(2)
            
            query = input("User > ")
            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
            query = query.lstrip()
            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
            query = query.rstrip()
            
            try:
                query = int(query)
                
                
                print("Bot > You want to check Detail Information about Patient {} Yes or No" .format(patient_name(query)))   
                query = input("User >")
                query = regrex(query)
                if (query=="exit"):
                    flag = False
                if (query.casefold()==Yes_check.casefold()):
                    print("Bot > Please provide your Search query again for confirmation")
                    query = input("User >")
                    query = regrex(query)
                    if (query == "exit"):
                        flag = False
                    else:
                        Bestmatch_1 = bestrecommend.getContext_bestrecommend(query)
                        for response in Bestmatch_1:
                            print("Bot > {}".format(response))
                            print(" ")
                        print("Bot > None of the Above") 
                        print(" ")
                        print("Bot > Above are the matching query,Select the suitable query")
                        print("Bot > Enter the best suitable query")
                        query = input("User >")
                        query = query.lstrip()
                        query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
                        query = query.rstrip()
                        query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
                        query = regrex(query)
                        if (query=="exit"):
                            flag = False
                    
                        elif(query == "None of the Above"):
                            print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                            break
                    
                        else:
                            answer = respond.sql_answer(query)
                            print("Please provide query to search again or type exit to end the Conversation")    
                
              
                elif (query.casefold()==No_check.casefold()):
                    print("bot > Sorry, I am currently unavailable to find query")
                    print("Please provide query to search again or type exit to end the Conversation")
            except:
                print("Bot > You are providing incorrect input")
                
                
            

    
    

User > hi


Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > aditya sonkar death


Bot > Paient Informations are very Sensitive Information to check the information 
 Please Provide the Patient id


User >  1


...

User > yes


Bot > Please provide your Search query again for confirmation


User > 


Bot > None of the Above
 
Bot > Above are the matching query,Select the suitable query
Bot > Enter the best suitable query


User > exit


_____________

_______________

_______________________

_________________

____________________________

__________

### Working on the patient id logic not found issue

### 2 issues
<ol>
    <li>id not found (resolved)</li>
    <li> Exact mapping on query with id(resolved)</li>
    </ol>

In [45]:
Doctors = ["Piyush Singh","Soumya","Mayank Mishra","Deepshikha Mishra","Rishabh Yadav"]
Patients = ["Aditya Sonkar","Harsh Mishra","Yuvraj Sinha","Amit","Garima"]

In [97]:
flag = True
context = None
Bestmatch = None
reply = None

In [98]:
while flag:
    Yes_check = 'Yes'
    No_check = 'No'
    query =input("User >")
    query = query.strip()
    query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
    query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
    
    query = regrex(query)
    print("Bot > Hello, I am a bot design for Specific Purpose How can I help you?")
    time.sleep(1)
    print("Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating")
    time.sleep(1)
    print("Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor")
    time.sleep(1)
    print("Bot > Here is the list of Doctors {}".format(Doctors))
    time.sleep(1)
    print("Bot > Here is the list of Patients {}".format(Patients))
    time.sleep(1)
    print("Bot > Please Provide your Query for Search")
    query = input("User >")
    query = regrex(query)

    score,_,context = getContext_details(query)
    
    if len(query)==0:
        print("Bot > Please provide the Input")
    elif query=="exit":
        flag=False
    
    else:
        if(score<0.4):
            print("Bot > I am a bot design for Database related issues, Please Provide Suitable Query")
            
        elif(score>=1.0):
            _,best_match,_ = getContext_details(query)
            answer = respond.sql_answer(best_match)
            print("Bot > Thanks for Searching")
            
            
            
            
        elif(score>=0.4 and context == "Doctor"):
            print("Bot > You want to check Detail Information about Doctor {} Yes or No" .format(get_name1((query))))
            query =input("User > ")
            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
            query = regrex(query)
            if(query == "exit"):
                flag = False
            if (query.casefold()==Yes_check.casefold()):
                print("Bot > Please provide your search query again for confirmation")
                query =input("User >")
                query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?, ,")
                query = regrex(query)
                if(query == "exit"):
                    flag = False
                else:
                    Bestmatch = bestrecommend.getContext_bestrecommend(query)
                    for response in Bestmatch:
                        print("Bot > {} ".format(response))
                        print(" ")
                    print("Bot > None of the Above")
                    print(" ")
                    print("Bot > Above are the matching query,Select the suitable query")
                    print("Bot > Enter the best suitable query")
                    query = input("User >")
                    query = query.lstrip()
                    query = query.rstrip()
                    query = query.lstrip("@,<,>,#,$,%,%,^,&,*")
                    query = regrex(query)
                    if (query=="exit"):
                        flag=False
                    elif(query == "None of the Above"):
                        print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                        break
                    else:
                        respond.sql_answer(query)
                        time.sleep(2)
                        
                
                
            else:
                print("bot > Sorry, I am currently unavailable to find query")
                flag = False
                
        elif(score>=0.4 and context=="Patient"):
            print("Bot > Paient Informations are very Sensitive Information to check the information \n Please Provide the Patient id")
            time.sleep(2)
            
            query = input("User > ")
            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
            query = query.lstrip()
            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
            query = query.rstrip()
            pat_id = cur.execute("SELECT Patient_id FROM Patient")
            patient_id_lst = pat_id_lst.id_checker_lst(pat_id)
            try:
                query = int(query)
                if query in patient_id_lst:
                    print("Bot > You want to check Detail Information about Patient {} Yes or No" .format(patient_name(query)))   
                    query = input("User >")
                    query = regrex(query)
                    if (query=="exit"):
                        flag = False
                    if (query.casefold()==Yes_check.casefold()):
                        print("Bot > Please provide your Search query again for confirmation")
                        query = input("User >")
                        query = regrex(query)
                        if (query == "exit"):
                            flag = False
                        else:
                            Bestmatch_1 = bestrecommend.getContext_bestrecommend(query)
                            for response in Bestmatch_1:
                                print("Bot > {}".format(response))
                                print(" ")
                            print("Bot > None of the Above") 
                            print(" ")
                            print("Bot > Above are the matching query,Select the suitable query")
                            print("Bot > Enter the best suitable query")
                            query = input("User >")
                            query = query.lstrip()
                            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
                            query = query.rstrip()
                            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
                            query = regrex(query)
                            if (query=="exit"):
                                flag = False
                    
                            elif(query == "None of the Above"):
                                print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                                break
                    
                            else:
                                answer = respond.sql_answer(query)
                                print("Please provide query to search again or type exit to end the Conversation")    
                
              
                    elif (query.casefold()==No_check.casefold()):
                          print("bot > Sorry, I am currently unavailable to find query")
                          print("Please provide query to search again or type exit to end the Conversation")
                            
                else:
                    print("bot > You are providing wrong id")
                    
                            
            except:
                print("Bot > You are providing incorrect input")
                
                
            

    
    

User > hi


Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > illness issue with harsh


Bot > Paient Informations are very Sensitive Information to check the information 
 Please Provide the Patient id


User >  7


...

User > no


bot > Sorry, I am currently unavailable to find query
Please provide query to search again or type exit to end the Conversation


User > exit


Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > exit
